In [34]:
import pandas as pd
import numpy as np
import gzip
import json

In [36]:
data = []
with gzip.open('data/steam_user_info.json.gzip', 'r') as f:
    for line in f:
        data.append(json.loads(line))

df_steam_user_info = pd.json_normalize(data)

In [4]:
df_steam_games=pd.read_json('data/steam_games.json',lines=True)

In [37]:
df_steam_user_info.head()

,user_id,items_count,steam_id,user_url,item_id_x,item_name,playtime_forever,playtime_2weeks,review_content,item_id_y,sentiment_analysis,recommend
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38...","[Counter-Strike, Team Fortress Classic, Day of...","[6, 0, 7, 0, 0, 0, 0, 0, 4733, 1853, 333, 75, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",['Simple yet with great replayability. In my o...,"[1250, 22200, 43110]","[2, 2, 2]","[True, True, True]"
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[10, 80, 100, 300, 30, 40, 60, 240, 280, 360, ...","[Counter-Strike, Counter-Strike: Condition Zer...","[0, 0, 0, 220, 0, 0, 0, 62, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",['I know what you think when you see this titl...,"[251610, 227300, 239030]","[2, 0, 0]","[True, True, True]"
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[1200, 1230, 1280, 1520, 220, 320, 340, 360, 3...","[Red Orchestra: Ostfront 41-45, Mare Nostrum, ...","[923, 0, 0, 158, 1323, 0, 90, 0, 234, 113, 507...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[""A suitably punishing roguelike platformer. ...","[248820, 370360, 237930, 263360, 107200, 224500]","[2, 2, 2, 0, 2, 2]","[True, True, True, True, True, True]"
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[10, 20, 30, 40, 50, 60, 70, 130, 80, 100, 300...","[Counter-Strike, Team Fortress Classic, Day of...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 67, 2304, 0, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",None,None,None,None
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[300, 20, 50, 70, 130, 10, 30, 40, 60, 80, 100...","[Day of Defeat: Source, Team Fortress Classic,...","[1131, 89, 178, 108, 313, 93, 16, 4, 0, 13, 47...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",['This game... is so fun. The fight sequences ...,"[250320, 20920, 204100, 224600, 207610, 108710]","[2, 2, 2, 2, 0, 2]","[True, True, True, True, True, True]"


In [6]:
import pandas as pd

# Convertimos las columnas a enteros
df_steam_user_info['item_id_x'] = df_steam_user_info['item_id_x'].apply(lambda x: [int(i) for i in x])
df_steam_games['id'] = df_steam_games['id'].astype(int)

# Luego, "expandimos" la lista en la columna item_id_x para que cada elemento tenga su propia fila
expanded_users_info = df_steam_user_info.explode('item_id_x')

# Realizamos el left join
merged = pd.merge(expanded_users_info, df_steam_games, how='inner', left_on='item_id_x', right_on='id')

# Seleccionamos las columnas de output_steam_games que queremos en el diccionario
columns_to_keep = ['publisher', 'genres', 'app_name', 'title', 'url', 'release_date', 'tags', 'reviews_url', 'specs', 'price','developer']

# Creamos una nueva columna que es un diccionario
merged['dict_column'] = merged[columns_to_keep].apply(lambda row: row.to_dict(), axis=1)

# Eliminamos las columnas originales de output_steam_games
merged.drop(columns_to_keep, axis=1, inplace=True)



In [7]:
merged.columns

Index(['user_id', 'items_count', 'steam_id', 'user_url', 'item_id_x',
       'item_name', 'playtime_forever', 'playtime_2weeks', 'review_content',
       'item_id_y', 'sentiment_analysis', 'recommend', 'Unnamed: 0',
       'early_access', 'id', 'dict_column'],
      dtype='object')

In [8]:
merged["dict_column"][1]

{'publisher': 'Valve',
 'genres': ['Action'],
 'app_name': 'Counter-Strike',
 'title': 'Counter-Strike',
 'url': 'http://store.steampowered.com/app/10/CounterStrike/',
 'release_date': '2000-11-01',
 'tags': '[\'Action\', \'FPS\', \'Multiplayer\', \'Shooter\', \'Classic\', \'Team-Based\', \'Competitive\', \'First-Person\', \'Tactical\', "1990\'s", \'e-sports\', \'PvP\', \'Military\', \'Strategy\', \'Score Attack\', \'Survival\', \'Assassin\', \'1980s\', \'Ninja\', \'Tower Defense\']',
 'reviews_url': 'http://steamcommunity.com/app/10/reviews/?browsefilter=mostrecent&p=1',
 'specs': "['Multi-player', 'Valve Anti-Cheat enabled']",
 'price': 9.99,
 'developer': 'Valve'}

In [9]:
merged["dict_column"][0]

{'publisher': 'Valve',
 'genres': ['Action'],
 'app_name': 'Counter-Strike',
 'title': 'Counter-Strike',
 'url': 'http://store.steampowered.com/app/10/CounterStrike/',
 'release_date': '2000-11-01',
 'tags': '[\'Action\', \'FPS\', \'Multiplayer\', \'Shooter\', \'Classic\', \'Team-Based\', \'Competitive\', \'First-Person\', \'Tactical\', "1990\'s", \'e-sports\', \'PvP\', \'Military\', \'Strategy\', \'Score Attack\', \'Survival\', \'Assassin\', \'1980s\', \'Ninja\', \'Tower Defense\']',
 'reviews_url': 'http://steamcommunity.com/app/10/reviews/?browsefilter=mostrecent&p=1',
 'specs': "['Multi-player', 'Valve Anti-Cheat enabled']",
 'price': 9.99,
 'developer': 'Valve'}

In [10]:
merged.columns

Index(['user_id', 'items_count', 'steam_id', 'user_url', 'item_id_x',
       'item_name', 'playtime_forever', 'playtime_2weeks', 'review_content',
       'item_id_y', 'sentiment_analysis', 'recommend', 'Unnamed: 0',
       'early_access', 'id', 'dict_column'],
      dtype='object')

In [11]:
merged=merged.drop("Unnamed: 0",axis=1)

In [12]:
merged_ordenado=merged.sort_values(by='user_id')

In [13]:
df_agrupado = merged.groupby('user_id').agg({'item_id_x': list, 'id': list, 'dict_column': list}).reset_index()


In [14]:
merged['dict_column_new'] = merged.apply(lambda row: {'item_id_x': row['item_id_x'], 'id': row['id'], 'dict_column': row['dict_column']}, axis=1)

df_agrupado = merged.groupby('user_id').agg({'dict_column_new': lambda x: list(x)}).reset_index()

# Cambia el nombre de las columnas en df_agrupado para evitar conflictos al unir
df_agrupado.columns = ['user_id', 'dict_column']

# Une df_agrupado con df
df_final = pd.merge(merged, df_agrupado, on='user_id')

# Elimina las columnas originales y la columna 'dict_column_new'
df_final.drop(columns=['item_id_x', 'id', 'dict_column_x', 'dict_column_new'], inplace=True)

# Renombra la nueva columna
df_final.rename(columns={'dict_column_y': 'dict_column'}, inplace=True)


In [15]:
df_agrupado.sample(2)

,user_id,dict_column
29464,76561198078557312,"[{'item_id_x': 304930, 'id': 304930, 'dict_col..."
61324,lsleon,"[{'item_id_x': 10, 'id': 10, 'dict_column': {'..."


In [16]:
print(df_agrupado.columns.tolist())

['user_id', 'dict_column']


In [17]:

df_agrupado['item_id_x'] = df_agrupado['dict_column'].apply(lambda x: [dic['item_id_x'] for dic in x])
df_agrupado['id'] = df_agrupado['dict_column'].apply(lambda x: [dic['id'] for dic in x])


In [18]:

columnas = ['user_id', 'item_id_x', 'id', 'dict_column']
df_agrupado = df_agrupado[columnas]


In [19]:
df_agrupado

,user_id,item_id_x,id,dict_column
0,--000--,"[1250, 35420, 550, 105600, 9480, 104700, 730, ...","[1250, 35420, 550, 105600, 9480, 104700, 730, ...","[{'item_id_x': 1250, 'id': 1250, 'dict_column'..."
1,--ace--,"[620, 113200, 730, 250900, 238460, 303390, 400...","[620, 113200, 730, 250900, 238460, 303390, 400...","[{'item_id_x': 620, 'id': 620, 'dict_column': ..."
2,--ionex--,"[550, 10180, 620, 105600, 730, 51100, 224260, ...","[550, 10180, 620, 105600, 730, 51100, 224260, ...","[{'item_id_x': 550, 'id': 550, 'dict_column': ..."
3,-2SV-vuLB-Kg,"[730, 200510, 303390, 318430, 104200, 48000, 2...","[730, 200510, 303390, 318430, 104200, 48000, 2...","[{'item_id_x': 730, 'id': 730, 'dict_column': ..."
4,-404PageNotFound-,"[240, 7670, 409710, 4560, 35140, 550, 10180, 8...","[240, 7670, 409710, 4560, 35140, 550, 10180, 8...","[{'item_id_x': 240, 'id': 240, 'dict_column': ..."
...,...,...,...,...
68530,zzonci,"[2400, 2420, 216150]","[2400, 2420, 216150]","[{'item_id_x': 2400, 'id': 2400, 'dict_column'..."
68531,zzoptimuszz,"[104700, 225840, 215080, 407530, 230410, 21512...","[104700, 225840, 215080, 407530, 230410, 21512...","[{'item_id_x': 104700, 'id': 104700, 'dict_col..."
68532,zzydrax,"[240, 730, 4000, 252490, 24240, 273110, 304930...","[240, 730, 4000, 252490, 24240, 273110, 304930...","[{'item_id_x': 240, 'id': 240, 'dict_column': ..."
68533,zzyfo,"[10, 240, 24740, 550, 8190, 33930, 17410, 9480...","[10, 240, 24740, 550, 8190, 33930, 17410, 9480...","[{'item_id_x': 10, 'id': 10, 'dict_column': {'..."


In [20]:
df_filtrado = df_agrupado[df_agrupado['user_id'] == 'zzonci']
df_filtrado

,user_id,item_id_x,id,dict_column
68530,zzonci,"[2400, 2420, 216150]","[2400, 2420, 216150]","[{'item_id_x': 2400, 'id': 2400, 'dict_column'..."


In [21]:
df_steam_user_info.head(2)

,user_id,items_count,steam_id,user_url,item_id_x,item_name,playtime_forever,playtime_2weeks,review_content,item_id_y,sentiment_analysis,recommend
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38...","[Counter-Strike, Team Fortress Classic, Day of...","[6, 0, 7, 0, 0, 0, 0, 0, 4733, 1853, 333, 75, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",['Simple yet with great replayability. In my o...,"[1250, 22200, 43110]","[2, 2, 2]","[True, True, True]"
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[10, 80, 100, 300, 30, 40, 60, 240, 280, 360, ...","[Counter-Strike, Counter-Strike: Condition Zer...","[0, 0, 0, 220, 0, 0, 0, 62, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",['I know what you think when you see this titl...,"[251610, 227300, 239030]","[2, 0, 0]","[True, True, True]"


In [22]:

df_final = pd.merge(df_steam_user_info,df_agrupado, on='user_id',how='left')


In [23]:

df_filtrado = df_final[df_final['user_id'] == 'helvisavillon']
df_filtrado

,user_id,items_count,steam_id,user_url,item_id_x_x,item_name,playtime_forever,playtime_2weeks,review_content,item_id_y,sentiment_analysis,recommend,item_id_x_y,id,dict_column
602,helvisavillon,2,76561198074228269,http://steamcommunity.com/id/helvisavillon,"[205790, 372000]","[Dota 2 Test, Tree of Savior (English Ver.)]","[0, 9799]","[0, 5527]",None,None,None,None,[372000],[372000],"[{'item_id_x': 372000, 'id': 372000, 'dict_col..."


In [24]:
df_final=df_final.drop(columns="items_count")
df_final['item_id_x_x'] = df_final['item_id_x_y'].where(df_final['item_id_x_y'].notna(), df_final['item_id_x_x'])
df_final=df_final.drop(columns="item_id_x_y")

In [25]:

nuevos_nombres = {
    'user_id': 'user_id',
    'steam_id': 'steam_id',
    'user_url': 'user_url',
    'item_id_x_x': 'user_item_id',
    'item_name': 'played_item_name',
    'playtime_forever': 'playtime_forever',
    'playtime_2weeks': 'playtime_2weeks',
    'review_content': 'review_content',
    'item_id_y': 'recomended_item_id',
    'sentiment_analysis': 'sentiment_analysis',
    'recommend': 'recommend',
    'id': 'id',
    'dict_column': 'games_info',
}
df_final.rename(columns=nuevos_nombres, inplace=True)

In [26]:
df_final.columns

Index(['user_id', 'steam_id', 'user_url', 'user_item_id', 'played_item_name',
       'playtime_forever', 'playtime_2weeks', 'review_content',
       'recomended_item_id', 'sentiment_analysis', 'recommend', 'id',
       'games_info'],
      dtype='object')

In [27]:
df_filtrado = df_final[df_final['user_id'] == 'helvisavillon']
df_filtrado

,user_id,steam_id,user_url,user_item_id,played_item_name,playtime_forever,playtime_2weeks,review_content,recomended_item_id,sentiment_analysis,recommend,id,games_info
602,helvisavillon,76561198074228269,http://steamcommunity.com/id/helvisavillon,[372000],"[Dota 2 Test, Tree of Savior (English Ver.)]","[0, 9799]","[0, 5527]",None,None,None,None,[372000],"[{'item_id_x': 372000, 'id': 372000, 'dict_col..."


In [28]:
df_final

,user_id,steam_id,user_url,user_item_id,played_item_name,playtime_forever,playtime_2weeks,review_content,recomended_item_id,sentiment_analysis,recommend,id,games_info
0,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38...","[Counter-Strike, Team Fortress Classic, Day of...","[6, 0, 7, 0, 0, 0, 0, 0, 4733, 1853, 333, 75, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",['Simple yet with great replayability. In my o...,"[1250, 22200, 43110]","[2, 2, 2]","[True, True, True]","[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38...","[{'item_id_x': 10, 'id': 10, 'dict_column': {'..."
1,js41637,76561198035864385,http://steamcommunity.com/id/js41637,"[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38...","[Counter-Strike, Counter-Strike: Condition Zer...","[0, 0, 0, 220, 0, 0, 0, 62, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",['I know what you think when you see this titl...,"[251610, 227300, 239030]","[2, 0, 0]","[True, True, True]","[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38...","[{'item_id_x': 10, 'id': 10, 'dict_column': {'..."
2,evcentric,76561198007712555,http://steamcommunity.com/id/evcentric,"[220, 320, 340, 360, 380, 400, 420, 13250, 105...","[Red Orchestra: Ostfront 41-45, Mare Nostrum, ...","[923, 0, 0, 158, 1323, 0, 90, 0, 234, 113, 507...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[""A suitably punishing roguelike platformer. ...","[248820, 370360, 237930, 263360, 107200, 224500]","[2, 2, 2, 0, 2, 2]","[True, True, True, True, True, True]","[220, 320, 340, 360, 380, 400, 420, 13250, 105...","[{'item_id_x': 220, 'id': 220, 'dict_column': ..."
3,Riot-Punch,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38...","[Counter-Strike, Team Fortress Classic, Day of...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 67, 2304, 0, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",None,None,None,None,"[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38...","[{'item_id_x': 10, 'id': 10, 'dict_column': {'..."
4,doctr,76561198002099482,http://steamcommunity.com/id/doctr,"[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38...","[Day of Defeat: Source, Team Fortress Classic,...","[1131, 89, 178, 108, 313, 93, 16, 4, 0, 13, 47...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",['This game... is so fun. The fight sequences ...,"[250320, 20920, 204100, 224600, 207610, 108710]","[2, 2, 2, 2, 0, 2]","[True, True, True, True, True, True]","[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38...","[{'item_id_x': 10, 'id': 10, 'dict_column': {'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71230,76561198320136420,76561198320136420,http://steamcommunity.com/profiles/76561198320...,"[41300, 104700, 212070, 225840, 319630, 273350...","[TrackMania Nations Forever, America's Army 3,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",None,None,None,None,"[41300, 104700, 212070, 225840, 319630, 273350...","[{'item_id_x': 41300, 'id': 41300, 'dict_colum..."
71231,ArkPlays7,76561198321542621,http://steamcommunity.com/id/ArkPlays7,"[730, 218230, 346110, 407530]","[PlanetSide 2, Counter-Strike: Global Offensiv...","[0, 4557, 623, 0]","[0, 1698, 0, 0]",None,None,None,None,"[730, 218230, 346110, 407530]","[{'item_id_x': 730, 'id': 730, 'dict_column': ..."
71232,76561198323066619,76561198323066619,http://steamcommunity.com/profiles/76561198323...,[417860],"[CS:GO Player Profiles, CS:GO Player Profiles:...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",None,None,None,None,[417860],"[{'item_id_x': 417860, 'id': 417860, 'dict_col..."
71233,76561198326700687,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[218230, 273350, 104900, 244590, 252150, 33524...","[TrackMania Nations Forever, Bloodline Champio...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [29]:
import pandas as pd

# Define una función para extraer el contenido de 'dict_column'
def extract_dict(row):
    if isinstance(row, dict) and 'dict_column' in row:
        # Crea un nuevo diccionario que incluye 'item_id_x' e 'id'
        new_dict = {'item_id_x': row['item_id_x'], 'id': row['id']}
        # Combina el nuevo diccionario con el contenido de 'dict_column'
        new_dict.update(row['dict_column'])
        return new_dict
    else:
        return row

# Aplica la función a cada elemento en la columna 'games_info'
df_final['games_info'] = df_final['games_info'].apply(lambda x: [extract_dict(i) for i in x] if isinstance(x, list) else x)


In [30]:
games_info = df_final["games_info"]

# Imprime los primeros 10 elementos de games_info
for i in range(3):
    print(games_info[i])


[{'item_id_x': 10, 'id': 10, 'publisher': 'Valve', 'genres': ['Action'], 'app_name': 'Counter-Strike', 'title': 'Counter-Strike', 'url': 'http://store.steampowered.com/app/10/CounterStrike/', 'release_date': '2000-11-01', 'tags': '[\'Action\', \'FPS\', \'Multiplayer\', \'Shooter\', \'Classic\', \'Team-Based\', \'Competitive\', \'First-Person\', \'Tactical\', "1990\'s", \'e-sports\', \'PvP\', \'Military\', \'Strategy\', \'Score Attack\', \'Survival\', \'Assassin\', \'1980s\', \'Ninja\', \'Tower Defense\']', 'reviews_url': 'http://steamcommunity.com/app/10/reviews/?browsefilter=mostrecent&p=1', 'specs': "['Multi-player', 'Valve Anti-Cheat enabled']", 'price': 9.99, 'developer': 'Valve'}, {'item_id_x': 20, 'id': 20, 'publisher': 'Valve', 'genres': ['Action'], 'app_name': 'Team Fortress Classic', 'title': 'Team Fortress Classic', 'url': 'http://store.steampowered.com/app/20/Team_Fortress_Classic/', 'release_date': '1999-04-01', 'tags': '[\'Action\', \'FPS\', \'Multiplayer\', \'Classic\', \

In [31]:
df_final.sample(3)

,user_id,steam_id,user_url,user_item_id,played_item_name,playtime_forever,playtime_2weeks,review_content,recomended_item_id,sentiment_analysis,recommend,id,games_info
66135,76561198092711304,76561198092711304,http://steamcommunity.com/profiles/76561198092...,"[41300, 346900, 230410, 224260, 304930, 301520...","[Dota 2 Test, Warframe, No More Room in Hell, ...","[0, 1898, 358, 0, 4610, 580, 56, 1, 1381, 1045...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",None,None,None,None,"[41300, 346900, 230410, 224260, 304930, 301520...","[{'item_id_x': 41300, 'id': 41300, 'publisher'..."
13235,76561198070677788,76561198070677788,http://steamcommunity.com/profiles/76561198070...,"[550, 730, 218230, 238460, 273350, 4000, 63380...","[Garry's Mod, Left 4 Dead 2, Left 4 Dead 2 Bet...","[60926, 2250, 0, 111354, 0, 1621, 11088, 1883,...","[0, 0, 0, 560, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","['HOLA ,EL JUEGO LA VERDAD ES BUENO PERO DECEP...",[9900],[1],[True],"[550, 730, 218230, 238460, 273350, 4000, 63380...","[{'item_id_x': 550, 'id': 550, 'publisher': 'V..."
29710,22109203,76561198004484134,http://steamcommunity.com/id/22109203,"[300, 240, 320, 340, 1250, 35420, 550, 33930, ...","[Counter-Strike: Source, Day of Defeat: Source...","[152220, 0, 6, 665, 331, 25326, 1290, 0, 1710,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",None,None,None,None,"[300, 240, 320, 340, 1250, 35420, 550, 33930, ...","[{'item_id_x': 300, 'id': 300, 'publisher': 'V..."


In [32]:

df_final.to_parquet('final.parquet.gzip', compression='gzip')